In [3]:
import  requests
import json

app_id = '7f3799d9'
app_key = '07d636d3ecf38d32b2ffbf299a13e871'
language = 'en'
word_id = 'ego'
url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/'  + language + '/'  + word_id.lower()


#url Normalized frequency
urlFR = 'https://od-api.oxforddictionaries.com:443/api/v2/stats/frequency/word/'  + language + '/?corpus=nmc&lemma=' + word_id.lower()
r = requests.get(url, headers = {'app_id' : app_id, 'app_key' : app_key})
#print("code {}\n".format(r.status_code))
print("text \n" + r.text)
print("json \n" + json.dumps(r.json()))

text 
{
    "id": "ego",
    "metadata": {
        "operation": "retrieve",
        "provider": "Oxford University Press",
        "schema": "RetrieveEntry"
    },
    "results": [
        {
            "id": "ego",
            "language": "en-gb",
            "lexicalEntries": [
                {
                    "derivatives": [
                        {
                            "id": "egoless",
                            "text": "egoless"
                        }
                    ],
                    "entries": [
                        {
                            "etymologies": [
                                "early 19th century: from Latin, literally ‘I’"
                            ],
                            "senses": [
                                {
                                    "definitions": [
                                        "a person's sense of self-esteem or self-importance"
                                    ],
                        